In [71]:
import json
import time
import requests
import urllib.parse
import urllib.request
import warnings
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
import datetime
# remove warnings
import multiprocessing

warnings.filterwarnings('ignore')

# Get Fire Data

In [64]:
base_url = "https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/"

start = time.perf_counter()
print('=' * 90)
print('handling {} '.format(base_url))

# Get record extract limit
url_string = base_url + "?f=json"
print("\nGetting record extract limit", url_string)
j = urllib.request.urlopen(url_string)
js = json.load(j)
max_records_count = int(js["maxRecordCount"])
print(("Record extract limit: %s" % max_records_count))
max_records_count = min(max_records_count, 800)

# get count of objects
count_query = "query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&returnCountOnly=true&outSR=4326&f=json"
url_string = base_url + count_query
print("\nGetting count of objects", url_string)
j = urllib.request.urlopen(url_string)
js = json.load(j)
num_of_records = js['count']
print(("Number of target records: %s" % num_of_records))

print("\nGathering records…")
gather_query = "query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&outSR=4326&f=json"
url_string = base_url + gather_query
resp = requests.get(url_string, verify=False)
print(url_string)
data = resp.json()

print("\nProcessing data...")
for feat in data['features']:
    geo = {'type':'polygon', 'coordinates':feat['geometry']['rings']}
    feat['geometry'] = geo
    feat['properties'] = feat['attributes']
    del feat['attributes']
fires = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4269')
fires = fires.loc[fires['geometry'].is_valid, :]
columns_lower = { col: col.lower() for col in fires.columns}
fires = fires.rename(columns=columns_lower)
fires['alarm_date'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in fires['alarm_date']]
fires['cont_date'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in fires['cont_date']]
fires.set_index['objectid']

end = time.perf_counter()
print('=' * 90)
print('Number of requests: {}'.format(1))
print(f'Finished in {round(end - start, 2)} second(s)')

handling https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/ 

Getting record extract limit https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/?f=json
Record extract limit: 2000

Getting count of objects https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&returnCountOnly=true&outSR=4326&f=json
Number of target records: 306

Gathering records…
https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_Fire_Perimeters/FeatureServer/0/query?where=%20(YEAR_%20%3D%202022%20OR%20YEAR_%20%3D%209999)%20&outFields=*&outSR=4326&f=json

Processing data...
Number of requests: 1
Finished in 2.01 second(s)


# Get Treatment Data

In [86]:
urls = ["https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/0",
        "https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/1",
        "https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/2"]

def fetch_all_features(base_url):
    start = time.perf_counter()
    print('=' * 90)
    print('handling {} '.format(base_url))
    
    # Get record extract limit
    url_string = base_url + "?f=json"
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    max_records_count = int(js["maxRecordCount"])
    print(("Record extract limit: %s" % max_records_count))
    
    max_records_count = min(max_records_count, 800)
    
    # Get object ids of features
    fields = "*"
    where = "1=1"
    url_string = base_url + "/query?where={}&returnIdsOnly=true&f=json".format(where)
    j = urllib.request.urlopen(url_string)
    js = json.load(j)
    id_field = js["objectIdFieldName"]
    id_list = js["objectIds"]
    id_list.sort()
    num_of_records = len(id_list)
    print(("Number of target records: %s" % num_of_records))
    
    print("Gathering records…")
    features_list = []
    
    def load_features(urlstring, return_dict):
        succeed = False
        while not succeed:
            try:
                resp = requests.get(urlstring, verify=False)
                data = resp.json()
                gdf = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4269')
                gdf = gdf.loc[gdf['geometry'].is_valid, :]
                return_dict[urlstring] = gdf
                succeed = True
            except:
                print ('Failed to load {}'.format(urlstring))
    processes = []
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    request_number = 0
    for i in range(0, num_of_records, max_records_count):
        request_number += 1
        to_rec = i + (max_records_count - 1)
        if to_rec > num_of_records:
            to_rec = num_of_records - 1
        from_id = id_list[i]
        to_id = id_list[to_rec]
        where = "{} >= {} and {} <= {}".format(id_field, from_id, id_field, to_id)
        print("  {}: {}".format(request_number, where))
        url_string = base_url + "/query?where={}&returnGeometry=true&outFields={}&f=geojson".format(where, fields)
    
        p = multiprocessing.Process(target=load_features, args=[url_string, return_dict])
        p.start()
        processes.append(p)
    
    for p in processes:
        p.join()
    p.close()
    
    for url in return_dict.keys():
        features_list.append(return_dict[url])
    treats = pd.concat(features_list)
    treats['activity_end'] = [datetime.datetime.fromtimestamp(ms/1000.0) for ms in treats['activity_end']]
    print ('Concatenating finished');
    end = time.perf_counter()
    print('-' * 50)
    print('Number of requests: {}'.format(request_number))
    print(f'Finished in {round(end - start, 2)} second(s)')
    return treats
    
treats = [fetch_all_features(url) for url in urls]

handling https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/0 
Record extract limit: 2000
Number of target records: 807
Gathering records…
  1: objectid >= 1 and objectid <= 800
  2: objectid >= 801 and objectid <= 807
Concatenating finished
--------------------------------------------------
Number of requests: 2
Finished in 2.42 second(s)
handling https://gsal.sig-gis.com/server/rest/services/Hosted/ITS_Dashboard_Feature_Layer/FeatureServer/1 
Record extract limit: 2000
Number of target records: 22014
Gathering records…
  1: objectid >= 1 and objectid <= 800
  2: objectid >= 801 and objectid <= 1600
  3: objectid >= 1601 and objectid <= 2400
  4: objectid >= 2401 and objectid <= 3200
  5: objectid >= 3201 and objectid <= 4000
  6: objectid >= 4001 and objectid <= 4800
  7: objectid >= 4801 and objectid <= 5600
  8: objectid >= 5601 and objectid <= 6400
  9: objectid >= 6401 and objectid <= 7200
  10: objectid >= 7201 and objectid <= 8000
  1

# Work With the Data

In [ ]:
def get_poly_treatment_intersections(objectid):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(str(fire['geometry']))
    poly_treats = treats[2]
    if fires.crs != poly_treats.crs:
        poly_treats = poly_treats.to_crs(fires.crs)
    #find treatments before fire that intersect with fire
    print("finding relevant treatment areas...")
    intersection_idxs = []
    for idx_t, treat_row in poly_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_poly = ogr.CreateGeometryFromWkt(str(treat_row['geometry']))
            if treat_poly.Intersects(fire_poly):
                intersection_idxs.append(idx_t)
    #get intersections
    print("found {0} intersections".format(len(intersection_idxs)))
    print("finding areas of intersection...")
    poly_treats_relevant = poly_treats.loc[intersection_idxs]
    intersections = []
    for idx_t, treat_row in poly_treats_relevant.iterrows():
        treat_poly = ogr.CreateGeometryFromWkt(str(treat_row['geometry']))
        intersections.append(treat_poly.Intersection(fire_poly))
    return intersections

def get_line_treatment_intersections(objectid, buffer_distance):
    fire = fires.loc[objectid]
    alarm_date = fire['alarm_date']
    fire_poly = ogr.CreateGeometryFromWkt(str(fire['geometry']))
    lp_treats = pd.concat(treats[:2])
    if fires.crs != lp_treats.crs:
        lp_treats = lp_treats.to_crs(fires.crs)
    #find treatments before fire that intersect with fire
    print("finding relevant treatment areas...")
    intersection_idxs = []
    for idx_t, treat_row in lp_treats.iterrows():
        if (treat_row['activity_end'] < alarm_date):
            treat_thing = ogr.CreateGeometryFromWkt(str(treat_row['geometry']))
            treat_geom = treat_thing.Buffer(buffer_distance)
            if treat_geom.Intersects(fire_poly):
                intersection_idxs.append(idx_t)
    print("found {0} intersections".format(len(intersection_idxs)))
    #get intersections
    print("finding areas of intersection...")
    lp_treats_relevant = lp_treats.loc[intersection_idxs]
    intersections = []
    for idx_t, treat_row in lp_treats_relevant.iterrows():
        treat_thing = ogr.CreateGeometryFromWkt(str(treat_row['geometry']))
        treat_geom = treat_thing.Buffer(buffer_distance)
        intersections.append(treat_geom.Intersection(fire_poly))
    return intersections
    

In [ ]:
objectid=301
buffer_distance=20
get_line_treatment_intersections(objectid, buffer_distance)